In [122]:
import sys, os, platform
import h5py, tarfile
import numpy as np
from pandas import Series, DataFrame


In [123]:
# h5filepath = 'e:/AShareSample/ETF/RFrames_SH510050.h5'
h5filepath = 'e:/temp/SH510050/RFrames_SH510050.h5'
h5file = h5py.File(h5filepath, 'r')
h5file.keys()

<KeysViewHDF5 ['ReplayFrame:DQN_Cnn1Dx4.1548_3.F0001', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0002', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0003', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0004', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0005', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0006', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0007', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0008', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0009', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0010', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0011', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0012', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0013', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0014', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0015', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0016', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0017', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0018', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0019', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0020', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0021', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0022', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0023', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0024', 'ReplayFrame:DQN_Cnn1Dx4.

In [124]:
framesInHd5 = []
for name in h5file.keys():
    if 'ReplayFrame:' == name[:len('ReplayFrame:')] :
        framesInHd5.append(name)

#if len(framesInHd5)>3:
#    del framesInHd5[0]
#    del framesInHd5[-1]

framesInHd5

['ReplayFrame:DQN_Cnn1Dx4.1548_3.F0001',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0002',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0003',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0004',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0005',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0006',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0007',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0008',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0009',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0010',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0011',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0012',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0013',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0014',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0015',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0016',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0017',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0018',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0019',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0020',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0021',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0022',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0023',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0024',
 'ReplayFrame:DQ

In [126]:
fname = framesInHd5[0]
fname

'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0001'

In [131]:
H5_COLS = 'state,action,reward,next_state,done'
samplePool = {
            'state':[],
            'action':[],
            'reward':[],
            'next_state':[],
            'done':[],
        }

frame = h5file[fname]
statebatch = np.array(frame['state'].value)
actionbatch = np.array(frame['action'].value)
# rewardbatch = np.array(frame['reward'].value)
#for col in H5_COLS.split(',') :
#    incrematal = list(frame[col].value)
#    samplePerFrame = len(incrematal)
#    samplePool[col] += incrematal
statebatch = statebatch.reshape(statebatch.shape[0],int(statebatch.shape[1] /4),4)
actionseq = ['H' if i[0]>0.5 else 'B' if i[1]>0.5 else 'S' for i in actionbatch ]
actionseq

['S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'S',
 'H',
 'H',
 'S',
 'H',
 'H',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'S',
 'S',
 'B',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S'

In [6]:
idxWhere =np.where(rewardbatch == np.max(rewardbatch))[0][0]
idxWhere

4966

In [7]:
# rewardbatch[idxWhere-10:idxWhere+100]

array([5.0000000e-01, 5.0000000e-01, 5.0000000e-01, 5.0000000e-01,
       7.9045124e+00, 5.0000000e-01, 5.0000000e-01, 5.0000000e-01,
       9.9999998e-03, 5.0000000e-01, 9.4877640e+01, 5.0000000e-01,
       5.0000000e-01, 5.0000000e-01, 1.1780336e+01, 5.0000000e-01,
       5.0000000e-01, 5.0000000e-01, 5.0000000e-01, 5.0000000e-01,
       5.0000000e-01, 5.0000000e-01, 5.0000000e-01, 7.8640790e+00,
       7.8578949e+00, 5.0000000e-01, 1.1780336e+01, 5.0000000e-01,
       5.0000000e-01, 7.8424778e+00, 7.8363280e+00, 3.9123466e+00,
       3.9108143e+00, 5.0000000e-01, 5.0000000e-01, 5.0000000e-01,
       5.0000000e-01, 5.0000000e-01, 5.0000000e-01, 5.0000000e-01,
       7.8548069e+00, 5.0000000e-01, 3.9246476e+00, 3.9231057e+00,
       5.0000000e-01, 5.0000000e-01, 1.0000000e-01, 1.1780336e+01,
       5.0000000e-01, 5.0000000e-01, 5.0000000e-01, 3.9215651e+00,
       5.0000000e-01, 7.8455563e+00, 5.0000000e-01, 5.0000000e-01,
       1.5696618e+01, 7.8332567e+00, 5.0000000e-01, 5.0000000e

In [203]:
from pandas import set_option
set_option('display.max_rows', 260)

In [208]:
idxDay, idx1m, idx5m, idx1d = 0, 1,31,(31+96)

#idxWhere = np.where(actionseq=='B')[0][5]
lenR =260
idxWhere =3500
rangeStart, rangeEnd=idxWhere, idxWhere+lenR
rangeStart, rangeEnd

(3500, 3760)

In [209]:
stateRange = statebatch[rangeStart:rangeEnd]
actionRange = actionbatch[rangeStart:rangeEnd]
actionseq = ['H' if i[0]>0.5 else 'B' if i[1]>0.5 else 'S' for i in actionRange ]
state1=[[r[idxDay][0],r[idxDay][1],r[idxDay][2],r[idxDay][3]/60,r[idx1m][0],r[idx1m][1],r[idx1m][2],r[idx1m][3],
         r[idx5m][0],r[idx5m][1],r[idx5m][2],r[idx5m][3],r[idx1d][0],r[idx1d][1],r[idx1d][2],r[idx1d][3]
        ]  for r in stateRange ]
act_state = [[actionseq[i]] + state1[i] for i in range(len(actionseq))]
DataFrame(act_state,columns='act,mon,day,wday,min,P1m,V1m,H1m,L1m,P5m,V5m,H5m,L5m,P1d,V1d,H1d,L1d'.split(','))

,act,mon,day,wday,min,P1m,V1m,H1m,L1m,P5m,V5m,H5m,L5m,P1d,V1d,H1d,L1d
0,S,2.0,6.0,2.0,14.683333,1.005342,1.348495,1.005828,1.004371,1.004371,0.898992,1.009228,1.004371,1.0,1.0,1.004857,0.985915
1,S,2.0,6.0,2.0,14.716667,1.004857,0.819580,1.005342,1.004857,1.004371,0.898992,1.009228,1.004371,1.0,1.0,1.004857,0.985915
2,S,2.0,6.0,2.0,14.750000,1.005342,1.570202,1.005342,1.004857,1.004371,0.898992,1.009228,1.004371,1.0,1.0,1.004857,0.985915
3,S,2.0,6.0,2.0,14.766667,1.005342,1.822644,1.006314,1.005342,1.005342,1.479237,1.005342,1.004371,1.0,1.0,1.004857,0.985915
4,S,2.0,6.0,2.0,14.800000,1.005828,0.350923,1.005828,1.005342,1.005342,1.479237,1.005342,1.004371,1.0,1.0,1.004857,0.985915
5,S,2.0,6.0,2.0,14.833333,1.005828,1.499746,1.006314,1.005342,1.005342,1.479237,1.005342,1.004371,1.0,1.0,1.004857,0.985915
6,S,2.0,6.0,2.0,14.850000,1.005828,2.292014,1.006314,1.005342,1.005828,0.431133,1.006314,1.005342,1.0,1.0,1.004857,0.985915
7,S,2.0,6.0,2.0,14.883333,1.005342,1.328385,1.006314,1.004857,1.005828,0.431133,1.006314,1.005342,1.0,1.0,1.004857,0.985915
8,S,2.0,6.0,2.0,14.916667,1.005342,0.153177,1.005342,1.004857,1.005828,0.431133,1.006314,1.005342,1.0,1.0,1.004857,0.985915
9,S,2.0,6.0,2.0,14.933333,1.005828,0.958637,1.005828,1.004857,1.005342,0.621235,1.006314,1.004857,1.0,1.0,1.004857,0.985915


In [9]:
state = statebatch[idxWhere]
header = state[:1]
# pos = state[1:2]
# time = state[2:3]
KL1m = state[1:(1+30)]
KL5m = state[31:(31+96)]
KL1d = state[(31+96):]
state

array([[8.0000000e+00, 7.0000000e+00, 4.0000000e+00, 1.4390000e+03],
       [1.0000000e+00, 8.9405137e-01, 1.0000000e+00, 9.9960065e-01],
       [1.0000000e+00, 1.0568796e+00, 1.0003994e+00, 9.9960065e-01],
       ...,
       [6.1222047e-01, 6.1432016e-01, 6.1341852e-01, 6.0343450e-01],
       [6.0503197e-01, 3.1286505e-01, 6.0742813e-01, 6.0383385e-01],
       [6.0543132e-01, 4.9845192e-01, 6.0782748e-01, 6.0063899e-01]],
      dtype=float32)

In [10]:
DataFrame(header,columns=['mon','day','wday','min'])

,mon,day,wday,min
0,8.0,7.0,4.0,1439.0


In [11]:
DataFrame(KL1m,columns=['P','V','H','L'])

,P,V,H,L
0,1.000000,0.894051,1.000000,0.999601
1,1.000000,1.056880,1.000399,0.999601
2,1.000399,1.362891,1.000399,0.999201
3,0.999601,1.368510,1.000000,0.999201
4,0.999601,1.194974,1.000000,0.999201
5,0.999601,0.685383,1.000399,0.999201
6,1.000000,0.415610,1.000799,1.000000
7,1.000399,1.068163,1.000399,0.999601
8,1.001198,2.384727,1.001198,0.999201
9,0.999201,1.005111,0.999601,0.998403


In [12]:
DataFrame(KL5m,columns=['P','V','H','L'])

,P,V,H,L
0,1.000000,0.936466,1.000399,0.999201
1,0.999601,0.910777,1.001198,0.999201
2,0.998802,0.795186,0.998802,0.996805
3,0.998003,0.618880,0.999201,0.997204
4,0.998003,0.595571,0.998403,0.995208
5,0.995208,1.029987,0.996406,0.994010
6,0.994808,0.846238,0.996006,0.993610
7,0.992812,0.824504,0.996006,0.992412
8,0.997604,0.556625,0.998802,0.997204
9,0.997204,0.475193,1.000000,0.996805


In [13]:
DataFrame(KL1d,columns=['P','V','H','L'])

,P,V,H,L
0,1.000000,1.000000,1.007188,0.990415
1,0.986821,0.935674,1.001597,0.981629
2,0.994409,0.940908,1.012380,0.994409
3,1.014377,0.998506,1.014776,0.985623
4,0.989617,1.134910,0.990016,0.966454
5,0.983227,1.377497,0.996805,0.974441
6,0.986022,1.278790,1.017572,0.983227
7,1.011981,1.628088,1.014377,0.983227
8,0.992013,3.894834,1.010783,0.962460
9,0.996006,2.085086,1.084665,0.995607
